In [1]:
import json
import pandas as pd
import altair as alt
import numpy as np
import altair as alt
from vega_datasets import data
from altair import datum

def SystemC():
    # import csv file use the first 2000 rows of data 
    supterstore_df = pd.read_csv('Global_Superstore.csv', low_memory=False, nrows=3000, encoding='UTF-8')
    # standardized the Order Date
    supterstore_df['Order Date'] = pd.to_datetime(supterstore_df['Order Date'], format='%d-%m-%Y')

    # Brushing and linking condition, limit selection to x-axis (year) values
    brush = alt.selection_interval(encodings=['x'])

    # Bar chart, dynamic query histogram,  add the brushing and linking condition
    # with x axis shows the year and y axis shows the total number of orders in that year
    years = alt.Chart(supterstore_df).mark_bar(color='#1f77b4').add_selection(
        brush
    ).encode(
        alt.X('yearmonth(Order Date):T', title='Month-Year'),
        alt.Y('count():Q', title='Total Number of Orders')
    ).properties(
        # customize zhe title and subtitle 
        title={
        "text": ["Total Number of Orders by Month"],
        "subtitle": ["Select a range on the bar chart to filter the scatter plot."],
        "subtitleColor": "gray"
    },
        width=750,
        height=200,
    )

    # add an interactive line chart layer to show the trend
    line = alt.Chart(supterstore_df).mark_line(color='#FFA500',tension=0.7).encode(
        alt.X('yearmonth(Order Date):T'),
        alt.Y('count():Q', title='Total Number of Orders')
    )

    # Create the bubble chart, with sales for X axis, and profit for Y axis, it will shows all the orders by default
    bubble = alt.Chart(supterstore_df).mark_point(filled=True).encode(
        alt.X('Sales:Q', title='Sales (USD)'),
        alt.Y('Profit:Q', title='Profit (USD)'),
        # add differnt color and shapes for the market in order to show then clearly
        alt.Color('Market:N', title='Market'),
        alt.Shape('Market:N'),
        alt.Size('Profit:Q', scale=alt.Scale(range=[30, 700]), title='Profit Value'),
        # set the tooltip
        tooltip=['Market:N', alt.Tooltip('Sales:Q', title='Sales($)'),alt.Tooltip('Profit:Q', title='Profit($)'), 'Category:N', 'Sub-Category:N'],
        # change the opacity according to the selected range
        opacity=alt.condition(brush, alt.value(0.75), alt.value(0.05))
        ).properties(
        # customize zhe title and subtitle 
        title={
        "text": ["Profit and Sales by Market"],
        "subtitle": ["Hover over a data point to see more information."],
        "subtitleColor": "gray"
    },
        width=750,
        height=400,
    ).interactive()

    # vertially show the charts and add 30 unit of space
    chart = alt.vconcat(bubble,alt.layer(years, line)).properties(spacing=30)
    chart.save("SystemC.html")
    chart.show()

SystemC()

Displaying chart at http://localhost:55958/
